# Implementation of Byte Pair Encoding
Since we want to

### Packages

In [3]:
import matplotlib.pyplot as plt
from collections import Counter

sentence = "FloydHub is the fastest way to build, train and deploy deep learning models. Build deep learning models in the cloud. Train deep learning models."


## Add suffixes

In [66]:
import re
from collections import Counter
import pandas as pd

def calFrequency(text, char_table):
    # 1. Split the text into words and add a suffix "<w>" to each word
    words = text.split(" ")
    words_with_sfx = [word + "<w>" for word in words]

    # 2. Calculate word frequency using Counter
    word_freq = Counter(words_with_sfx)

    # 3. Calculate character frequency while treating '<w>' as a single character
    text_with_sfx = " ".join(words_with_sfx)  # Join words with suffix "<w>"

    # Create a regex pattern that treats each character in char_table as a single unit
    re_sub_list = '|'.join(re.escape(char) for char in char_table)
    print(re_sub_list)
    # Find all characters, treating '<w>' and other defined characters as units
    characters = re.findall(fr'({re_sub_list}|[^ ])', text_with_sfx)  # Use [^\s] to match non-whitespace characters
    char_freq = Counter(characters)

    # 4. Convert the word and character frequency into pandas DataFrames (tables)
    word_freq_table = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])
    char_freq_table = pd.DataFrame(char_freq.items(), columns=['Character', 'Frequency'])

    # 5. Sort the tables by frequency in descending order
    word_freq_table = word_freq_table.sort_values(by='Frequency', ascending=False)
    char_freq_table = char_freq_table.sort_values(by='Frequency', ascending=False)

    # 6. Return the tables
    return word_freq_table, char_freq_table

## Keep word table
until size of words table = 10

**Problem of this code: how can we decide the order of characters for pairs.**

In [73]:
dict_size = 10
char_table = ['<w>']
word_freq_table, char_freq_table = calFrequency(sentence, char_table)
print(char_freq_table)
i=0

# Expanding character pairs until the desired dictionary size is reached
while len(char_freq_table) != dict_size:

    char1, char2 = char_freq_table.iloc[1:3]["Character"]
    pair = char1 + char2 if char1<char2 else char2+char1
    print(f'Iter: {i} \n Best pair: {pair} \n Tokens: {char_freq_table} \n Numbers of tokens: {len(char_freq_table)}')

    if pair not in char_table:  # Avoid duplicates
        char_table.append(pair)  # Use add to update the set
        word_freq_table, char_freq_table = calFrequency(sentence, char_table)

    i+=1

    if(i>5):
        break
    

<w>
   Character  Frequency
8        <w>         24
13         e         16
4          d         12
1          l         11
19         n         10
9          i          9
15         a          8
2          o          7
11         t          6
10         s          6
18         r          5
6          u          4
20         p          4
21         g          3
23         .          3
22         m          3
3          y          3
7          b          2
12         h          2
24         B          1
25         c          1
0          F          1
17         ,          1
16         w          1
14         f          1
5          H          1
26         T          1
Iter: 0 
 Best pair: de 
 Tokens:    Character  Frequency
8        <w>         24
13         e         16
4          d         12
1          l         11
19         n         10
9          i          9
15         a          8
2          o          7
11         t          6
10         s          6
18         r          5
6 

## Correct version
1. We start by tokenizing the sentence into individual characters, converting everything to lowercase.
2. We then perform 10 merge operations (or fewer if there are no more pairs to merge).
3. In each iteration, we find the most frequent pair of adjacent tokens and merge them.
4. Finally, we print out the resulting vocabulary.

In [2]:
import re
from collections import defaultdict

def get_stats(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out


In [6]:

# Initial setup
sentence = "FloydHub is the fastest way to build, train and deploy deep learning models. Build deep learning models in the cloud. Train deep learning models."
words = sentence.split()
vocab = defaultdict(int)

# Tokenize the sentence
for word in words:
    tokenized_word = ' '.join(word.lower()) # FloydHub -> f l o y d h u b
    vocab[tokenized_word] += 1


num_merges = 20

for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(f"Merge #{i+1}: {best}")

print("\nFinal vocabulary:")
for word, freq in vocab.items():
    print(f"{word}: {freq}")

defaultdict(<class 'int'>, {('f', 'l'): 1, ('l', 'o'): 3, ('o', 'y'): 2, ('y', 'd'): 1, ('d', 'h'): 1, ('h', 'u'): 1, ('u', 'b'): 1, ('i', 's'): 1, ('t', 'h'): 2, ('h', 'e'): 2, ('f', 'a'): 1, ('a', 's'): 1, ('s', 't'): 2, ('t', 'e'): 1, ('e', 's'): 1, ('w', 'a'): 1, ('a', 'y'): 1, ('t', 'o'): 1, ('b', 'u'): 2, ('u', 'i'): 2, ('i', 'l'): 2, ('l', 'd'): 2, ('d', ','): 1, ('t', 'r'): 2, ('r', 'a'): 2, ('a', 'i'): 2, ('i', 'n'): 6, ('a', 'n'): 1, ('n', 'd'): 1, ('d', 'e'): 7, ('e', 'p'): 4, ('p', 'l'): 1, ('e', 'e'): 3, ('l', 'e'): 3, ('e', 'a'): 3, ('a', 'r'): 3, ('r', 'n'): 3, ('n', 'i'): 3, ('n', 'g'): 3, ('m', 'o'): 3, ('o', 'd'): 3, ('e', 'l'): 3, ('l', 's'): 3, ('s', '.'): 2, ('c', 'l'): 1, ('o', 'u'): 1, ('u', 'd'): 1, ('d', '.'): 1})
{'f l o y d h u b': 1, 'i s': 1, 't h e': 2, 'f a s t e s t': 1, 'w a y': 1, 't o': 1, 'b u i l d ,': 1, 't r a i n': 2, 'a n d': 1, 'de p l o y': 1, 'de e p': 3, 'l e a r n i n g': 3, 'm o de l s .': 2, 'b u i l d': 1, 'm o de l s': 1, 'i n': 1, 'c l